<a href="https://colab.research.google.com/github/harita-gr/ICTACK-AI-ML/blob/main/Assignment04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hyperparameter Tuning

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("/content/credit_card_default.csv")

### Step 1. Data Analysis

In [3]:
data.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [4]:
data.shape

(30000, 25)

In [5]:
data.columns

Index(['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_1',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default payment next month'],
      dtype='object')

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   ID                          30000 non-null  int64
 1   LIMIT_BAL                   30000 non-null  int64
 2   SEX                         30000 non-null  int64
 3   EDUCATION                   30000 non-null  int64
 4   MARRIAGE                    30000 non-null  int64
 5   AGE                         30000 non-null  int64
 6   PAY_1                       30000 non-null  int64
 7   PAY_2                       30000 non-null  int64
 8   PAY_3                       30000 non-null  int64
 9   PAY_4                       30000 non-null  int64
 10  PAY_5                       30000 non-null  int64
 11  PAY_6                       30000 non-null  int64
 12  BILL_AMT1                   30000 non-null  int64
 13  BILL_AMT2                   30000 non-null  int64
 14  BILL_A

In [7]:
data.isna().sum()

ID                            0
LIMIT_BAL                     0
SEX                           0
EDUCATION                     0
MARRIAGE                      0
AGE                           0
PAY_1                         0
PAY_2                         0
PAY_3                         0
PAY_4                         0
PAY_5                         0
PAY_6                         0
BILL_AMT1                     0
BILL_AMT2                     0
BILL_AMT3                     0
BILL_AMT4                     0
BILL_AMT5                     0
BILL_AMT6                     0
PAY_AMT1                      0
PAY_AMT2                      0
PAY_AMT3                      0
PAY_AMT4                      0
PAY_AMT5                      0
PAY_AMT6                      0
default payment next month    0
dtype: int64

### Step 2: Data Pre-processing

2.1 Handle missing values in columns `PAY_AMT6` and
`default payment next month `

In [8]:
data['PAY_AMT6'].value_counts()

PAY_AMT6
0        7173
1000     1299
2000     1295
3000      914
5000      808
         ... 
7329        1
6862        1
6525        1
11894       1
16080       1
Name: count, Length: 6939, dtype: int64

In [9]:
# Fill missing values with median
data['PAY_AMT6'].fillna(data['PAY_AMT6'].median(), inplace=True)

In [10]:
data['default payment next month'].value_counts()

default payment next month
0    23364
1     6636
Name: count, dtype: int64

In [11]:
# Fill missing values with mode
data['default payment next month'].fillna(data['default payment next month'].mode()[0], inplace=True)

Verifying result

In [12]:
data.isna().sum()

ID                            0
LIMIT_BAL                     0
SEX                           0
EDUCATION                     0
MARRIAGE                      0
AGE                           0
PAY_1                         0
PAY_2                         0
PAY_3                         0
PAY_4                         0
PAY_5                         0
PAY_6                         0
BILL_AMT1                     0
BILL_AMT2                     0
BILL_AMT3                     0
BILL_AMT4                     0
BILL_AMT5                     0
BILL_AMT6                     0
PAY_AMT1                      0
PAY_AMT2                      0
PAY_AMT3                      0
PAY_AMT4                      0
PAY_AMT5                      0
PAY_AMT6                      0
default payment next month    0
dtype: int64

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   ID                          30000 non-null  int64
 1   LIMIT_BAL                   30000 non-null  int64
 2   SEX                         30000 non-null  int64
 3   EDUCATION                   30000 non-null  int64
 4   MARRIAGE                    30000 non-null  int64
 5   AGE                         30000 non-null  int64
 6   PAY_1                       30000 non-null  int64
 7   PAY_2                       30000 non-null  int64
 8   PAY_3                       30000 non-null  int64
 9   PAY_4                       30000 non-null  int64
 10  PAY_5                       30000 non-null  int64
 11  PAY_6                       30000 non-null  int64
 12  BILL_AMT1                   30000 non-null  int64
 13  BILL_AMT2                   30000 non-null  int64
 14  BILL_A

## Step 3: Split the data

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X = data.drop(columns=['default payment next month'])  # Features
y = data['default payment next month']  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Verifying result

In [16]:
# Print the shapes of the training and testing sets
print("Training set shape:", X_train.shape, y_train.shape)
print("Testing set shape:", X_test.shape, y_test.shape)

Training set shape: (24000, 24) (24000,)
Testing set shape: (6000, 24) (6000,)


In [17]:
X.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
0,1,20000,2,2,1,24,2,2,-1,-1,...,689,0,0,0,0,689,0,0,0,0
1,2,120000,2,2,2,26,-1,2,0,0,...,2682,3272,3455,3261,0,1000,1000,1000,0,2000
2,3,90000,2,2,2,34,0,0,0,0,...,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000
3,4,50000,2,2,1,37,0,0,0,0,...,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000
4,5,50000,1,2,1,57,-1,0,-1,0,...,35835,20940,19146,19131,2000,36681,10000,9000,689,679


## Step 4: Perform Different Algorithms with hyperparameter tuning for each algorithm using grid search cross-validation

In [18]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

### 4.1 kNN

In [19]:
knn_params = {'n_neighbors': [3, 5, 7, 9]}  # Define the grid of hyperparameters
knn_grid = GridSearchCV(KNeighborsClassifier(), knn_params, cv=5)  # Initialize GridSearchCV
knn_grid.fit(X_train, y_train)  # Perform grid search
best_knn = knn_grid.best_estimator_  # Get the best estimator
accuracy_knn_tuned = knn_grid.best_score_  # Get the best cross-validation score

### 4.2 SVM

In [ ]:
svm_params = {'C': [0.1, 1, 10], 'gamma': [0.1, 0.01, 0.001], 'kernel': ['rbf', 'linear']}
svm_grid = GridSearchCV(SVC(), svm_params, cv=5)  # Initialize GridSearchCV
svm_grid.fit(X_train, y_train)  # Perform grid search
best_svm = svm_grid.best_estimator_  # Get the best estimator
accuracy_svm_tuned = svm_grid.best_score_  # Get the best cross-validation scoresvm_params = {'C': [0.1, 1, 10], 'gamma': [0.1, 0.01, 0.001], 'kernel': ['rbf', 'linear']}
svm_grid = GridSearchCV(SVC(), svm_params, cv=5)  # Initialize GridSearchCV
svm_grid.fit(X_train, y_train)  # Perform grid search
best_svm = svm_grid.best_estimator_  # Get the best estimator
accuracy_svm_tuned = svm_grid.best_score_  # Get the best cross-validation score

### 4.3 Gradient Boosting

In [21]:
gb_params = {'n_estimators': [50, 100, 200], 'max_depth': [3, 5, 7]}
gb_grid = GridSearchCV(GradientBoostingClassifier(), gb_params, cv=5)  # Initialize GridSearchCV
gb_grid.fit(X_train, y_train)  # Perform grid search
best_gb = gb_grid.best_estimator_  # Get the best estimator
accuracy_gb_tuned = gb_grid.best_score_  # Get the best cross-validation score

### 4.4 AdaBoost

In [22]:
ada_params = {'n_estimators': [50, 100, 200]}
ada_grid = GridSearchCV(AdaBoostClassifier(), ada_params, cv=5)  # Initialize GridSearchCV
ada_grid.fit(X_train, y_train)  # Perform grid search
best_ada = ada_grid.best_estimator_  # Get the best estimator
accuracy_ada_tuned = ada_grid.best_score_  # Get the best cross-validation score

### 4.5 XGBoost


In [23]:
xgb_params = {'n_estimators': [50, 100, 200], 'max_depth': [3, 5, 7], 'learning_rate': [0.1, 0.01, 0.001]}
xgb_grid = GridSearchCV(xgb.XGBClassifier(), xgb_params, cv=5)  # Initialize GridSearchCV
xgb_grid.fit(X_train, y_train)  # Perform grid search
best_xgb = xgb_grid.best_estimator_  # Get the best estimator
accuracy_xgb_tuned = xgb_grid.best_score_  # Get the best cross-validation score

In [28]:
print("Tuned hyperparameters and best cross-validation scores:")
print("kNN:", best_knn)
print("kNN best score:", accuracy_knn_tuned)
print("SVM:", best_svm)
print("SVM best score:", accuracy_svm_tuned)
print("Gradient Boosting:", best_gb)
print("Gradient Boosting best score:", accuracy_gb_tuned)
print("AdaBoost:", best_ada)
print("AdaBoost best score:", accuracy_ada_tuned)
print("XGBoost:", best_xgb)
print("XGBoost best score:", accuracy_xgb_tuned)

Tuned hyperparameters and best cross-validation scores:
kNN: KNeighborsClassifier(n_neighbors=9)
kNN best score: 0.7626249999999999
Gradient Boosting: GradientBoostingClassifier(n_estimators=50)
Gradient Boosting best score: 0.821875
AdaBoost: AdaBoostClassifier(n_estimators=100)
AdaBoost best score: 0.8168749999999999
XGBoost: XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimato